# Modeling

In [16]:
# This file will train the models.
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle
from sklearn.feature_selection import SelectFromModel

from statistics import multimode
from cmath import nan


import seaborn as sns
pd.set_option("display.max_rows", None, "display.max_columns", None)

from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import cross_val_score


# Functions

In [17]:
def models(X_train,Y_train):
      
  #Using Logistic Regression Algorithm to the Training Set
  from sklearn.linear_model import LogisticRegression
  log = LogisticRegression(random_state = 0)
  log.fit(X_train, Y_train)  
  
  #Using KNeighborsClassifier Method of neighbors class to use Nearest Neighbor algorithm
  from sklearn.tree import DecisionTreeClassifier
  tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
  tree.fit(X_train, Y_train)

  #Using RandomForestClassifier method of ensemble class to use Random Forest Classification algorithm
  from sklearn.ensemble import RandomForestClassifier
  forest = RandomForestClassifier()
  forest.fit(X_train, Y_train)
  
  # Niave
  from sklearn.naive_bayes import GaussianNB
  naive = GaussianNB()
  naive.fit(X_train, Y_train)

  #linearSVM
  from sklearn.svm import LinearSVC
  svm = LinearSVC()
  svm.fit(X_train,Y_train)



  #print model accuracy on the training data.
  print('[0]Logistic Regression Training Accuracy:', log.score(X_train, Y_train))
  print('[1]Decsion Tree Classifier Training Accuracy:', tree.score(X_train, Y_train))
  print('[2]Random Forest Classifier Training Accuracy:', forest.score(X_train, Y_train))
  print('[3]naive Classifier Training Accuracy:', naive.score(X_train, Y_train))
  print('[4]linearSVM Classifier Training Accuracy:', svm.score(X_train, Y_train))

  
  return log, tree, forest,naive,svm

In [18]:
def testSet(set):
    test = set.loc[set.Protest == -1]
    test.drop('Protest', axis=1, inplace = True)
    train = set.loc[set.Protest != -1]
    
    num_features = 14
    X = train.iloc[:, 1:num_features].values 
    Y = train.iloc[:, 0].values
    
    # Split the dataset into 80% Training set and 20% Testing set
    from sklearn.model_selection import train_test_split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

    #Get and train all of the models
    model = models(X_train,Y_train)

    #Print Confusion Matrices
    for i in range(len(model)):
        cm = confusion_matrix(Y_test, model[i].predict(X_test)) 
        #extracting TN, FP, FN, TP
        TN, FP, FN, TP = confusion_matrix(Y_test, model[i].predict(X_test)).ravel()
        print('Model[{}] Testing Accuracy = "{} !"'.format(i,  (TP + TN) / (TP + TN + FN + FP)))

In [19]:
def reccommended_features(X_train,Y_train):
    forest = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    select_forest = SelectFromModel(estimator=forest)
    new_X = select_forest.fit(X,Y) 
    selected_feat= X.columns[(select_forest.get_support())]
    len(selected_feat)

    print(selected_feat)

In [20]:
def get_grid(X_train, Y_train):
    from sklearn.pipeline import Pipeline
    from sklearn.model_selection import GridSearchCV
    import numpy as np

    max_features_range = np.arange(1,5,1)
    n_estimators_range = np.arange(10,180,10)
    random_state_range = np.arange(0,5,1)

    param_grid = dict(max_features=max_features_range, 
                    n_estimators=n_estimators_range)

    # Inputting random state here gives better results than when inputting later. 
    rf = RandomForestClassifier(random_state = 0, criterion = 'entropy')
    grid = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
    grid.fit(X_train, Y_train)
    print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))
    return(grid)

In [21]:
def grid_plot(grid):
    import pandas as pd
    grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["Accuracy"])],axis=1)
    grid_contour = grid_results.groupby(['max_features','n_estimators']).mean()
    grid_reset = grid_contour.reset_index()
    grid_reset.columns = ['max_features', 'n_estimators',  'Accuracy']
    grid_pivot = grid_reset.pivot('max_features', 'n_estimators')
    x = grid_pivot.columns.levels[1].values
    y = grid_pivot.index.values
    z = grid_pivot.values
    import plotly.graph_objects as go
    # X and Y axes labels
    layout = go.Layout(
                xaxis=go.layout.XAxis(
                title=go.layout.xaxis.Title(
                text='n_estimators')
                ),
                yaxis=go.layout.YAxis(
                title=go.layout.yaxis.Title(
                text='max_features') 
                ) )

    fig = go.Figure(data= [go.Surface(z=z, y=y, x=x)], layout=layout )
    fig.update_layout(title='Hyperparameter tuning',
                    scene = dict(
                        xaxis_title='n_estimators',
                        yaxis_title='max_features',
                        zaxis_title='Accuracy'),
                    autosize=False,
                    width=800, height=800,
                    margin=dict(l=65, r=50, b=65, t=90))
    fig.show()

# Main

In [22]:
training = pd.read_csv('DATA/super_data (1).csv')
training = training.drop('Unnamed: 0', axis = 1)
training = training[['Protest', 'NuTweets', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'd10', 'average', 'likes', 'followers', 'retweets', 'replies',
       'sub', 'pol']]
training = training.loc[((training['Protest']== 1) & (training['NuTweets'] > 20) )| (training['Protest']==0)]
training.head(1)

,Protest,NuTweets,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,average,likes,followers,retweets,replies,sub,pol
0,0,6,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.5,0.5,1.0,0.6,6.833333,4030.0,4.0,0.666667,0.429545,0.259975


# Testing

In [23]:
testSet(training)

c:\Users\Joseph\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[0]Logistic Regression Training Accuracy: 0.6832204665161776
[1]Decsion Tree Classifier Training Accuracy: 1.0
[2]Random Forest Classifier Training Accuracy: 1.0
[3]naive Classifier Training Accuracy: 0.6598946576373212
[4]linearSVM Classifier Training Accuracy: 0.6606471030850264
Model[0] Testing Accuracy = "0.6216216216216216 !"
Model[1] Testing Accuracy = "0.7237237237237237 !"
Model[2] Testing Accuracy = "0.7207207207207207 !"
Model[3] Testing Accuracy = "0.6336336336336337 !"
Model[4] Testing Accuracy = "0.6246246246246246 !"


c:\Users\Joseph\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# Suggestion maker

In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

In [25]:
X = training.drop('Protest',axis = 1)
Y = training['Protest']

In [26]:
reccommended_features(X,Y)

Index(['NuTweets', 'average', 'retweets'], dtype='object')


# Hyperparams

In [27]:
training.head(1)

,Protest,NuTweets,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,average,likes,followers,retweets,replies,sub,pol
0,0,6,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.5,0.5,1.0,0.6,6.833333,4030.0,4.0,0.666667,0.429545,0.259975


In [28]:
# X and Y as defined above
# Split the dataset into 80% Training set and 20% Testing set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(training.drop('Protest',axis=1), training['Protest'], test_size = 0.2, random_state = 0)

In [30]:
rf = RandomForestClassifier(max_features=4, n_estimators=100, random_state = 0)
rf.fit(X_train, Y_train)
rf.score(X_test, Y_test)
Y_pred = rf.predict(X_test)
print(accuracy_score(Y_pred, Y_test))
print(rf.score(X_test,Y_test))

0.7357357357357357
0.7357357357357357


In [31]:
grid = get_grid(X_train,Y_train)

The best parameters are {'max_features': 3, 'n_estimators': 160} with a score of 0.77


In [32]:
grid_plot(grid)

# Dropping some features

In [33]:
training.head(1)

,Protest,NuTweets,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,average,likes,followers,retweets,replies,sub,pol
0,0,6,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.5,0.5,1.0,0.6,6.833333,4030.0,4.0,0.666667,0.429545,0.259975


In [34]:
training.columns

Index(['Protest', 'NuTweets', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'd10', 'average', 'likes', 'followers', 'retweets', 'replies',
       'sub', 'pol'],
      dtype='object')

In [35]:
subs = []

# 1
temp = training[['Protest', 'NuTweets', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'd10', 'average', 'likes', 'followers', 'retweets', 'replies',
       'sub', 'pol']]
subs.append(temp)
# 2
temp = training[['Protest', 'NuTweets', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'd10', 'average']]
subs.append(temp)
# 3
temp = training[['Protest', 'NuTweets',  'average', 'likes', 'followers', 'retweets', 'replies',
       'sub', 'pol']]
subs.append(temp)
# 4
temp = training[['Protest', 'NuTweets', 'd7', 'average']]
subs.append(temp)
# 5
temp = training[['Protest', 'NuTweets',  'd3', 'd4', 'd5', 'd6', 'd7', 'average']]
subs.append(temp)
# 1
temp = training[['Protest', 'NuTweets', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'd10', 'average', 'likes', 'followers', 'retweets','sub', 'pol']]
subs.append(temp)



In [36]:


for sub in subs:
    X_train, X_test, Y_train, Y_test = train_test_split(sub.drop('Protest',axis=1), training['Protest'], test_size = 0.2)
    rf.fit(X_train, Y_train)
    rf.score(X_test, Y_test)
    Y_pred = rf.predict(X_test)
    print(accuracy_score(Y_pred, Y_test))

0.7387387387387387
0.7207207207207207
0.7507507507507507


ValueError: max_features must be in (0, n_features]

# Creating the Fully trained model. 

In [41]:
X = training.drop('Protest',axis=1)
Y = training['Protest']

In [43]:
randomforest = RandomForestClassifier(n_estimators = 160, max_features = 3, random_state = 0, criterion = 'entropy')
randomforest = randomforest.fit(X,Y)

In [51]:
def save_model(model, filename):
    pickle.dump(model, open('DATA/'+filename, 'wb'))

def load_model(filename):
    loaded_model = pickle.load(open('DATA/'+filename, 'rb'))
    return(loaded_model)



In [49]:
save_model(randomforest,'RF')